# ПРОГНОЗИРОВАНИЕ ВОЗМОЖНОСТИ ДЕПРЕССИИ

В этой тетради мы изучаем данные о психическом здоровье из опроса, чтобы проанализировать и разработать модель, которая поможет предсказать вероятность депрессии.

Это тетрадь для начинающих, чтобы понять и познакомиться с МО и прогнозным анализом, и мы сосредоточимся на построении модели XGBoost для обучения и подгонки наших данных. В общем, получайте удовольствие и проявляйте творческий подход!

### Сначала мы импортируем все необходимые библиотеки.

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

### Сохранение и изменение данных

Теперь мы читаем исходные данные, устанавливаем нашу цель и выбираем наши предикторы. Мы также разделяем категориальные и числовые столбцы и выбираем столбцы с низкой кардинальностью для использования в эффективном прогнозировании.

In [ ]:
# Loading all data 

X_full = pd.read_csv("/kaggle/input/playground-series-s4e11/train.csv", index_col='id')
X_test_full = pd.read_csv("/kaggle/input/playground-series-s4e11/test.csv", index_col= 'id')

# Separate target and predictors

X_full.dropna(axis=0, subset=['Depression'], inplace=True)
y = X_full.Depression
X_full.drop(['Depression'], axis=1, inplace=True)

# Split test data

X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_full, y, train_size=0.8, 
                                                                test_size=0.2, 
                                                                random_state=0)

# Select only low cardinality categorical columns and numerical columns for analysis and prediction

categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

Let's take a peek at our training data.

In [ ]:
X_train.head()

### Предварительная обработка

Проще всего использовать конвейеры для поддержания чистого и проверяемого кода. Выполняет необходимые подстановки, кодирование.

In [ ]:
# Numerical transformer
num_trans = SimpleImputer(strategy = 'constant')

# Categorical Transformer
cat_trans = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ohen', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_trans, numerical_cols),
        ('cat', cat_trans, categorical_cols)
    ])

# One-hot encode the data
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_test = pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

### Модель обучения и подбора

Мы разрабатываем модель путем тонкой настройки параметров и обучаем модель с использованием наших обучающих данных. Затем мы проверяем ее с использованием наших данных проверки для прогнозирования и вычисляем среднюю абсолютную ошибку с набором проверки y.

In [ ]:
# Define model

model  = XGBRegressor(n_estimators = 900, early_stopping_rounds = 4, learning_rate = 0.03)

# Fit the model 

model.fit(X_train, y_train, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)

# Get predictions
predictions = model.predict(X_valid)

# Calculate MAE
mae = mean_absolute_error(predictions, y_valid)

print("Mean Absolute Error:" , mae)

In [ ]:
# Save test predictions to file
test_preds = model.predict(X_test)
output = pd.DataFrame({'id': X_test.index,
                       'Depression': test_preds.round()})
output.to_csv('submission.csv', index=False)
print("And we're done! Good luck to everybody !!")